# Análisis de Datos - Machine Learning

Este notebook contiene el análisis completo de datos con modelos de machine learning.

## Pasos del análisis:
1. Carga de datos
2. Conversión a JSON y conteo por cuenta
3. Análisis Exploratorio de Datos (EDA)
4. Filtrado de datos (Tarifa NBO y Rentabilizacion)
5. Preparación de datos
6. Entrenamiento de modelos (Regresión Logística, Random Forest, XGBoost)
7. Validación cruzada
8. Grid Search para árboles de decisión
9. Comparación y selección del mejor modelo


## Paso 1: Importación de librerías


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
import warnings
import json
import pickle
from datetime import datetime
import os

warnings.filterwarnings('ignore')

print("✓ Librerías importadas correctamente")


## Paso 2: Carga de datos


In [ ]:
archivo = "Total_Mes_Act_Datos completos CORREGIDO.csv"
output_dir = 'resultados'
os.makedirs(output_dir, exist_ok=True)

print("Cargando datos...")
try:
    df = pd.read_csv(
        archivo, 
        low_memory=False,
        encoding='utf-8',
        on_bad_lines='skip',
        sep=',',
        quotechar='"'
    )
    print(f"✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
except Exception as e:
    print(f"Error: {e}")
    print("Intentando con encoding alternativo...")
    df = pd.read_csv(
        archivo,
        low_memory=False,
        encoding='latin-1',
        on_bad_lines='skip',
        sep=';',
        quotechar='"'
    )
    print(f"✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")

df.head()


## Paso 3: Conversión a JSON y Conteo por Cuenta


In [ ]:
print("Convirtiendo datos a JSON (muestra de 100,000 registros)...")
max_rows_json = min(100000, len(df))
df_muestra = df.head(max_rows_json)

ruta_json = os.path.join(output_dir, 'datos_completos.json')
df_muestra.to_json(
    ruta_json,
    orient='records',
    date_format='iso',
    indent=2,
    force_ascii=False
)
print(f"✓ Archivo JSON guardado en '{ruta_json}'")
print(f"Total de registros convertidos: {len(df_muestra)}")


In [ ]:
columna_cuenta = None

for col in df.columns:
    if 'cuenta' in col.lower():
        columna_cuenta = col
        break

if columna_cuenta is None:
    posibles = [col for col in df.columns if 'cuent' in col.lower() or 'account' in col.lower()]
    if posibles:
        columna_cuenta = posibles[0]

if columna_cuenta:
    print(f"Columna seleccionada: {columna_cuenta}")
    conteo = df[columna_cuenta].value_counts().reset_index()
    conteo.columns = ['Cuenta', 'Frecuencia']
    conteo = conteo.sort_values('Frecuencia', ascending=False)
    
    print(f"\nTotal de cuentas únicas: {len(conteo)}")
    print(f"\nTop 10 cuentas más frecuentes:")
    display(conteo.head(10))
    
    archivo_csv = os.path.join(output_dir, 'conteo_por_cuenta.csv')
    conteo.to_csv(archivo_csv, index=False, encoding='utf-8')
    print(f"\n✓ Conteo guardado en '{archivo_csv}'")
else:
    print("No se encontró columna 'cuenta'")


## Paso 4: Análisis Exploratorio de Datos (EDA)


In [ ]:
print("Información del dataset:")
print(f"Dimensiones: {df.shape}")
print(f"\nColumnas: {len(df.columns)}")
print(f"\nTipos de datos:")
print(df.dtypes.value_counts())

print(f"\nValores faltantes:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Valores_Faltantes': missing,
    'Porcentaje': missing_pct
})
missing_df = missing_df[missing_df['Valores_Faltantes'] > 0].sort_values('Porcentaje', ascending=False)

if len(missing_df) > 0:
    display(missing_df.head(20))
else:
    print("No hay valores faltantes")


## Paso 5: Identificación y Filtrado de Datos


In [ ]:
columna_tarifa = None
columna_target = None

for col in df.columns:
    if 'tarifa' in col.lower() and 'nbo' in col.lower():
        columna_tarifa = col
        break

if columna_tarifa is None:
    posibles = [col for col in df.columns if 'tarifa' in col.lower() or 'nbo' in col.lower()]
    if posibles:
        columna_tarifa = posibles[0]

for col in df.columns:
    if 'rentabilizacion' in col.lower() or 'rentabiliz' in col.lower():
        columna_target = col
        break

if columna_target is None:
    posibles = [col for col in df.columns if 'rent' in col.lower()]
    if posibles:
        columna_target = posibles[0]

print(f"Columna Tarifa NBO: {columna_tarifa}")
print(f"Columna Rentabilizacion: {columna_target}")

if columna_tarifa and columna_target:
    df_filtrado = df[df[columna_tarifa].notna()].copy()
    print(f"\nFilas después de filtrar por {columna_tarifa}: {len(df_filtrado)}")
    
    df_filtrado = df_filtrado[df_filtrado[columna_target].notna()].copy()
    print(f"Filas después de filtrar por {columna_target}: {len(df_filtrado)}")
    
    print(f"\nDistribución de la variable objetivo ({columna_target}):")
    display(df_filtrado[columna_target].value_counts())
else:
    print("\nError: No se encontraron las columnas necesarias")
    df_filtrado = None


## Paso 6: Preparación de Datos para Modelado


In [ ]:
if df_filtrado is not None and columna_target:
    y = df_filtrado[columna_target].copy()
    X = df_filtrado.drop(columns=[columna_target])
    
    columnas_numericas = X.select_dtypes(include=[np.number]).columns.tolist()
    columnas_categoricas = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f"Columnas numéricas: {len(columnas_numericas)}")
    print(f"Columnas categóricas: {len(columnas_categoricas)}")
    
    X_processed = X[columnas_numericas].copy()
    
    for col in columnas_categoricas:
        if X[col].nunique() < 50:
            le = LabelEncoder()
            X_processed[col] = le.fit_transform(X[col].astype(str).fillna('Missing'))
    
    X_processed = X_processed.fillna(X_processed.median())
    
    if isinstance(y.dtype, object) or y.dtype == 'object':
        le_target = LabelEncoder()
        y = le_target.fit_transform(y.astype(str))
    else:
        y = y.astype(int)
    
    print(f"\nShape final: X={X_processed.shape}, y={y.shape}")
    print(f"Clases en y: {np.unique(y)}")
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\n✓ Datos preparados:")
    print(f"  Train: {X_train.shape[0]} muestras")
    print(f"  Test: {X_test.shape[0]} muestras")
else:
    print("Error: No se pueden preparar los datos")


## Paso 7: Entrenamiento de Modelos

Dividimos el flujo en subpasos utilizando lecturas por bloques de 50.000 filas para monitorear el progreso y reutilizar los artefactos generados.


## Paso 8: Grid Search con muestreo estratificado


In [ ]:
if 'X_processed' not in globals() or 'y' not in globals():
    raise RuntimeError('Es necesario ejecutar el Paso 7.1.1 antes de continuar.')

n_muestra = min(300000, len(X_processed))
print(f"Tomando muestra estratificada de {n_muestra} registros para grid search...", flush=True)

X_muestra, _, y_muestra, _ = train_test_split(
    X_processed,
    y,
    train_size=n_muestra,
    random_state=42,
    stratify=y
)

X_train_grid, X_test_grid, y_train_grid, y_test_grid = train_test_split(
    X_muestra,
    y_muestra,
    test_size=0.2,
    random_state=42,
    stratify=y_muestra
)

print(f"  Train: {X_train_grid.shape[0]} muestras", flush=True)
print(f"  Test: {X_test_grid.shape[0]} muestras", flush=True)

cv_grid = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

print('\nGrid Search - Random Forest', flush=True)
parametros_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)
modelo_grid_rf = GridSearchCV(
    rf_base,
    parametros_rf,
    cv=cv_grid,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)
modelo_grid_rf.fit(X_train_grid, y_train_grid)

resultado_grid_rf = {
    'best_params': modelo_grid_rf.best_params_,
    'best_score': float(modelo_grid_rf.best_score_)
}

with open(os.path.join(output_dir, 'grid_rf.json'), 'w', encoding='utf-8') as archivo_rf:
    json.dump(resultado_grid_rf, archivo_rf, indent=2)

print(f"✓ Mejor combinación RF: {resultado_grid_rf['best_params']} (score {resultado_grid_rf['best_score']:.4f})", flush=True)

print('\nGrid Search - XGBoost', flush=True)
parametros_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.8, 1.0]
}

xgb_base = XGBClassifier(random_state=42, n_jobs=-1, eval_metric='logloss')
modelo_grid_xgb = GridSearchCV(
    xgb_base,
    parametros_xgb,
    cv=cv_grid,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)
modelo_grid_xgb.fit(X_train_grid, y_train_grid)

resultado_grid_xgb = {
    'best_params': modelo_grid_xgb.best_params_,
    'best_score': float(modelo_grid_xgb.best_score_)
}

with open(os.path.join(output_dir, 'grid_xgb.json'), 'w', encoding='utf-8') as archivo_xgb:
    json.dump(resultado_grid_xgb, archivo_xgb, indent=2)

print(f"✓ Mejor combinación XGBoost: {resultado_grid_xgb['best_params']} (score {resultado_grid_xgb['best_score']:.4f})", flush=True)
print('\n✓ Paso 8 completado', flush=True)


## Paso 9: Resumen de resultados de Grid Search


In [ ]:
ruta_grid_rf = os.path.join(output_dir, 'grid_rf.json')
ruta_grid_xgb = os.path.join(output_dir, 'grid_xgb.json')

with open(ruta_grid_rf, 'r', encoding='utf-8') as archivo_rf:
    resumen_rf = json.load(archivo_rf)

with open(ruta_grid_xgb, 'r', encoding='utf-8') as archivo_xgb:
    resumen_xgb = json.load(archivo_xgb)

print('Resultados guardados de Grid Search:', flush=True)
print(f"  Random Forest -> score CV: {resumen_rf['best_score']:.4f} | parámetros: {resumen_rf['best_params']}", flush=True)
print(f"  XGBoost       -> score CV: {resumen_xgb['best_score']:.4f} | parámetros: {resumen_xgb['best_params']}", flush=True)

if 'modelo_grid_rf' in globals() and 'modelo_grid_xgb' in globals():
    print('\nEvaluación rápida en el conjunto de prueba retenido de la muestra:', flush=True)
    pred_rf = modelo_grid_rf.best_estimator_.predict(X_test_grid)
    pred_xgb = modelo_grid_xgb.best_estimator_.predict(X_test_grid)
    print(f"  Random Forest -> accuracy test: {accuracy_score(y_test_grid, pred_rf):.4f}", flush=True)
    print(f"  XGBoost       -> accuracy test: {accuracy_score(y_test_grid, pred_xgb):.4f}", flush=True)
else:
    print('\nPara evaluar en test es necesario volver a ejecutar el Paso 8 en esta sesión.', flush=True)


## Paso 10: Comparación y Conclusión


In [ ]:
ruta_grid_rf = os.path.join(output_dir, 'grid_rf.json')
ruta_grid_xgb = os.path.join(output_dir, 'grid_xgb.json')
ruta_cv_lr = os.path.join(output_dir, 'resultado_cv_lr.json')

faltantes = [ruta for ruta in [ruta_grid_rf, ruta_grid_xgb, ruta_cv_lr] if not os.path.exists(ruta)]
if faltantes:
    raise FileNotFoundError(f"No se encontraron los archivos requeridos: {faltantes}")

with open(ruta_grid_rf, 'r', encoding='utf-8') as archivo_rf:
    resumen_rf = json.load(archivo_rf)

with open(ruta_grid_xgb, 'r', encoding='utf-8') as archivo_xgb:
    resumen_xgb = json.load(archivo_xgb)

with open(ruta_cv_lr, 'r', encoding='utf-8') as archivo_lr:
    resumen_lr = json.load(archivo_lr)

print('COMPARACIÓN DE MODELOS', flush=True)
print('=' * 80, flush=True)
print('\nValidación cruzada - Regresión logística:', flush=True)
print(f"  Accuracy promedio: {resumen_lr['mean']:.4f} (+/- {resumen_lr['std'] * 2:.4f})", flush=True)

print('\nGrid Search - Modelos de árboles:', flush=True)
print(f"  Random Forest -> score CV: {resumen_rf['best_score']:.4f} | parámetros: {resumen_rf['best_params']}", flush=True)
print(f"  XGBoost       -> score CV: {resumen_xgb['best_score']:.4f} | parámetros: {resumen_xgb['best_params']}", flush=True)

mejor_arbol = 'RandomForest' if resumen_rf['best_score'] > resumen_xgb['best_score'] else 'XGBoost'

print('\n' + '=' * 80, flush=True)
print('CONCLUSIÓN', flush=True)
print('=' * 80, flush=True)
print(f"  Mejor modelo entre árboles (según Grid Search): {mejor_arbol}", flush=True)

conclusion = {
    'cv_lr_mean': resumen_lr['mean'],
    'cv_lr_std': resumen_lr['std'],
    'grid_rf': resumen_rf,
    'grid_xgb': resumen_xgb,
    'best_tree_model': mejor_arbol
}

ruta_conclusion = os.path.join(output_dir, 'conclusion_paso_10.json')
with open(ruta_conclusion, 'w', encoding='utf-8') as archivo_conclusion:
    json.dump(conclusion, archivo_conclusion, indent=2)

print(f"\n✓ Conclusión almacenada en: {ruta_conclusion}", flush=True)


## Paso 11: Análisis de Importancia de Variables y Visualizaciones

En este paso analizamos las variables más importantes usando el mejor modelo (XGBoost) y creamos visualizaciones del análisis.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_tree, plot_importance
import graphviz
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
from sklearn.tree import export_graphviz
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print('=' * 80)
print('PASO 11: ANÁLISIS DE IMPORTANCIA Y VISUALIZACIONES')
print('=' * 80)


In [ ]:
print('\nCargando datos preparados y resultados...', flush=True)

ruta_datos = os.path.join(output_dir, 'datos_preparados_7_1.pkl')
with open(ruta_datos, 'rb') as f:
    datos = pickle.load(f)

X_train_scaled = datos['X_train_scaled']
X_test_scaled = datos['X_test_scaled']
y_train = datos['y_train']
y_test = datos['y_test']

print(f'✓ Datos cargados: Train={X_train_scaled.shape[0]}, Test={X_test_scaled.shape[0]}', flush=True)

with open(os.path.join(output_dir, 'grid_xgb.json'), 'r', encoding='utf-8') as f:
    grid_xgb_params = json.load(f)

print(f'✓ Parámetros óptimos XGBoost cargados: {grid_xgb_params["best_params"]}', flush=True)


In [ ]:
print('\nEntrenando modelo XGBoost con parámetros óptimos...', flush=True)

xgb_optimizado = XGBClassifier(
    n_estimators=grid_xgb_params['best_params']['n_estimators'],
    max_depth=grid_xgb_params['best_params']['max_depth'],
    learning_rate=grid_xgb_params['best_params']['learning_rate'],
    subsample=grid_xgb_params['best_params']['subsample'],
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)

xgb_optimizado.fit(X_train_scaled, y_train)

y_pred_xgb = xgb_optimizado.predict(X_test_scaled)
y_pred_proba_xgb = xgb_optimizado.predict_proba(X_test_scaled)[:, 1]

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'✓ Modelo entrenado - Accuracy en test: {accuracy_xgb:.4f}', flush=True)

ruta_modelo_xgb = os.path.join(output_dir, 'modelo_xgb_optimizado.pkl')
with open(ruta_modelo_xgb, 'wb') as f:
    pickle.dump(xgb_optimizado, f)
print(f'✓ Modelo guardado en: {ruta_modelo_xgb}', flush=True)


### 11.1: Importancia de Características


In [ ]:
print('\nAnalizando importancia de características...', flush=True)

importancia = xgb_optimizado.feature_importances_
feature_names = [f'Feature_{i}' for i in range(len(importancia))]

df_importancia = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importancia
}).sort_values('Importance', ascending=False)

print(f'\nTop 20 características más importantes:', flush=True)
print(df_importancia.head(20).to_string(index=False), flush=True)

df_importancia.to_csv(os.path.join(output_dir, 'importancia_caracteristicas.csv'), index=False)
print(f'\n✓ Importancia guardada en CSV', flush=True)


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 16))

top_n = 20
df_top = df_importancia.head(top_n)

axes[0, 0].barh(range(len(df_top)), df_top['Importance'].values, color='steelblue')
axes[0, 0].set_yticks(range(len(df_top)))
axes[0, 0].set_yticklabels(df_top['Feature'].values)
axes[0, 0].set_xlabel('Importancia', fontsize=12, fontweight='bold')
axes[0, 0].set_title(f'Top {top_n} Características Más Importantes (XGBoost)', 
                     fontsize=14, fontweight='bold', pad=20)
axes[0, 0].invert_yaxis()
axes[0, 0].grid(axis='x', alpha=0.3)

axes[0, 1].bar(range(len(df_top)), df_top['Importance'].values, color='coral')
axes[0, 1].set_xticks(range(len(df_top)))
axes[0, 1].set_xticklabels(df_top['Feature'].values, rotation=45, ha='right')
axes[0, 1].set_ylabel('Importancia', fontsize=12, fontweight='bold')
axes[0, 1].set_title(f'Top {top_n} Características - Vista de Barras', 
                     fontsize=14, fontweight='bold', pad=20)
axes[0, 1].grid(axis='y', alpha=0.3)

importancia_acumulada = df_importancia['Importance'].cumsum()
axes[1, 0].plot(range(1, len(importancia_acumulada) + 1), importancia_acumulada.values, 
                marker='o', linewidth=2, markersize=4, color='green')
axes[1, 0].axhline(y=0.8, color='r', linestyle='--', label='80% de importancia')
axes[1, 0].set_xlabel('Número de Características', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Importancia Acumulada', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Importancia Acumulada de Características', 
                     fontsize=14, fontweight='bold', pad=20)
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

n_features_80 = len(importancia_acumulada[importancia_acumulada <= 0.8])
axes[1, 0].axvline(x=n_features_80, color='orange', linestyle='--', 
                   label=f'{n_features_80} features = 80%')
axes[1, 0].legend()

axes[1, 1].pie(df_top['Importance'].values, labels=df_top['Feature'].values, 
               autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title(f'Distribución de Importancia - Top {top_n}', 
                     fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
ruta_grafica = os.path.join(output_dir, 'importancia_caracteristicas.png')
plt.savefig(ruta_grafica, dpi=300, bbox_inches='tight')
print(f'✓ Gráfica de importancia guardada en: {ruta_grafica}', flush=True)
plt.show()

print(f'\nNúmero de características que explican el 80% de la importancia: {n_features_80}', flush=True)
print(f'Características clave (top {n_features_80}):', flush=True)
for i, row in df_importancia.head(n_features_80).iterrows():
    print(f"  {row['Feature']}: {row['Importance']:.6f}", flush=True)


### 11.2: Visualización de Árboles de Decisión


In [ ]:
print('\nVisualizando árboles de decisión individuales...', flush=True)

try:
    fig, axes = plt.subplots(2, 2, figsize=(30, 24))
    axes = axes.flatten()
    
    for i, ax in enumerate(axes[:4]):
        plot_tree(xgb_optimizado, num_trees=i, ax=ax, rankdir='LR')
        ax.set_title(f'Árbol de Decisión #{i+1} (XGBoost)', fontsize=12, fontweight='bold', pad=10)
    
    plt.tight_layout()
    ruta_arboles = os.path.join(output_dir, 'arboles_decision_xgb.png')
    plt.savefig(ruta_arboles, dpi=300, bbox_inches='tight')
    print(f'✓ Árboles de decisión guardados en: {ruta_arboles}', flush=True)
    plt.show()
    
except Exception as e:
    print(f'⚠ No se pudieron visualizar los árboles con plot_tree: {e}', flush=True)
    print('Intentando método alternativo...', flush=True)
    
    try:
        from xgboost import to_graphviz
        
        for i in range(min(3, xgb_optimizado.n_estimators)):
            graph = to_graphviz(xgb_optimizado, num_trees=i)
            ruta_arbol = os.path.join(output_dir, f'arbol_decision_{i+1}.dot')
            graph.save(ruta_arbol)
            print(f'✓ Árbol {i+1} guardado en formato DOT: {ruta_arbol}', flush=True)
    except Exception as e2:
        print(f'⚠ Método alternativo también falló: {e2}', flush=True)
        print('Los árboles están disponibles en el modelo pero requieren herramientas adicionales para visualización.', flush=True)


### 11.3: Matriz de Confusión y Métricas


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

cm = confusion_matrix(y_test, y_pred_xgb)
precision = precision_score(y_test, y_pred_xgb)
recall = recall_score(y_test, y_pred_xgb)
f1 = f1_score(y_test, y_pred_xgb)
roc_auc = roc_auc_score(y_test, y_pred_proba_xgb)

print('\n' + '=' * 80)
print('MÉTRICAS DEL MODELO XGBOOST')
print('=' * 80)
print(f'Accuracy:  {accuracy_xgb:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall:    {recall:.4f}')
print(f'F1-Score:  {f1:.4f}')
print(f'ROC-AUC:   {roc_auc:.4f}')
print('=' * 80)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0], 
            xticklabels=['Clase 0', 'Clase 1'], 
            yticklabels=['Clase 0', 'Clase 1'])
axes[0].set_xlabel('Predicción', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Real', fontsize=12, fontweight='bold')
axes[0].set_title('Matriz de Confusión', fontsize=14, fontweight='bold', pad=20)

tn, fp, fn, tp = cm.ravel()
axes[1].bar(['Verdaderos\nNegativos', 'Falsos\nPositivos', 'Falsos\nNegativos', 'Verdaderos\nPositivos'],
            [tn, fp, fn, tp], color=['green', 'orange', 'red', 'blue'], alpha=0.7)
axes[1].set_ylabel('Cantidad', fontsize=12, fontweight='bold')
axes[1].set_title('Desglose de la Matriz de Confusión', fontsize=14, fontweight='bold', pad=20)
axes[1].grid(axis='y', alpha=0.3)

for i, v in enumerate([tn, fp, fn, tp]):
    axes[1].text(i, v + max([tn, fp, fn, tp]) * 0.01, str(v), 
                ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
ruta_confusion = os.path.join(output_dir, 'matriz_confusion_xgb.png')
plt.savefig(ruta_confusion, dpi=300, bbox_inches='tight')
print(f'\n✓ Matriz de confusión guardada en: {ruta_confusion}', flush=True)
plt.show()

reporte = classification_report(y_test, y_pred_xgb, output_dict=True)
df_reporte = pd.DataFrame(reporte).transpose()
df_reporte.to_csv(os.path.join(output_dir, 'reporte_clasificacion_xgb.csv'))
print('✓ Reporte de clasificación guardado en CSV', flush=True)


### 11.4: Curvas ROC y Precision-Recall


In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_xgb)
roc_auc = auc(fpr, tpr)

precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba_xgb)
pr_auc = auc(recall_curve, precision_curve)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC curve (AUC = {roc_auc:.4f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
axes[0].set_xlabel('Tasa de Falsos Positivos', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Tasa de Verdaderos Positivos', fontsize=12, fontweight='bold')
axes[0].set_title('Curva ROC', fontsize=14, fontweight='bold', pad=20)
axes[0].legend(loc="lower right", fontsize=11)
axes[0].grid(alpha=0.3)

axes[1].plot(recall_curve, precision_curve, color='blue', lw=2,
             label=f'Precision-Recall curve (AUC = {pr_auc:.4f})')
baseline = len(y_test[y_test==1]) / len(y_test)
axes[1].axhline(y=baseline, color='r', linestyle='--', 
                label=f'Baseline ({baseline:.4f})')
axes[1].set_xlabel('Recall', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Precision', fontsize=12, fontweight='bold')
axes[1].set_title('Curva Precision-Recall', fontsize=14, fontweight='bold', pad=20)
axes[1].legend(loc="lower left", fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
ruta_curvas = os.path.join(output_dir, 'curvas_roc_precision_recall.png')
plt.savefig(ruta_curvas, dpi=300, bbox_inches='tight')
print(f'✓ Curvas ROC y Precision-Recall guardadas en: {ruta_curvas}', flush=True)
plt.show()

print(f'\nÁrea bajo la curva ROC: {roc_auc:.4f}', flush=True)
print(f'Área bajo la curva Precision-Recall: {pr_auc:.4f}', flush=True)


### 11.5: Resumen de Variables Clave


In [ ]:
print('\n' + '=' * 80)
print('RESUMEN DE VARIABLES CLAVE PARA EL MODELO')
print('=' * 80)

n_variables_clave = min(15, len(df_importancia))
variables_clave = df_importancia.head(n_variables_clave)

print(f'\nLas {n_variables_clave} variables más importantes son:\n', flush=True)
for idx, row in variables_clave.iterrows():
    porcentaje = (row['Importance'] / df_importancia['Importance'].sum()) * 100
    print(f"  {idx+1:2d}. {row['Feature']:20s} - Importancia: {row['Importance']:.6f} ({porcentaje:.2f}%)", flush=True)

importancia_total_top = variables_clave['Importance'].sum()
porcentaje_total = (importancia_total_top / df_importancia['Importance'].sum()) * 100
print(f'\nEstas {n_variables_clave} variables explican el {porcentaje_total:.2f}% de la importancia total.', flush=True)

resumen_variables = {
    'variables_clave': variables_clave['Feature'].tolist(),
    'importancia_total': float(importancia_total_top),
    'porcentaje_explicado': float(porcentaje_total),
    'total_variables': len(df_importancia),
    'variables_seleccionadas': n_variables_clave
}

ruta_resumen = os.path.join(output_dir, 'resumen_variables_clave.json')
with open(ruta_resumen, 'w', encoding='utf-8') as f:
    json.dump(resumen_variables, f, indent=2)

print(f'\n✓ Resumen de variables clave guardado en: {ruta_resumen}', flush=True)
print('\n✓ Paso 11 completado exitosamente', flush=True)


## Paso 12: Visualización de Árboles de Decisión (Estructura)

Se generan imágenes de la estructura de los árboles de decisión:
- Intento 1: Árboles internos del modelo XGBoost (si hay soporte de Graphviz).
- Intento 2: Árbol surrogado (sklearn) entrenado sobre las variables más importantes, exportado como PNG.


In [ ]:
print('Generando visualizaciones de árboles...', flush=True)

# Asegurar df_importancia y nombres de features
ruta_importancias = os.path.join(output_dir, 'importancia_caracteristicas.csv')
if 'df_importancia' not in globals():
    if os.path.exists(ruta_importancias):
        df_importancia = pd.read_csv(ruta_importancias)
    else:
        raise RuntimeError('No se encuentra df_importancia. Ejecute el Paso 11 primero.')

# Intento 1: Visualizar árboles de XGBoost directamente (requiere Graphviz)
exitos_xgb = False
try:
    from xgboost import plot_tree as xgb_plot_tree
    fig, axes = plt.subplots(2, 2, figsize=(30, 24))
    axes = axes.flatten()
    for i, ax in enumerate(axes[:4]):
        xgb_plot_tree(xgb_optimizado, num_trees=i, ax=ax, rankdir='LR')
        ax.set_title(f'Estructura - Árbol XGBoost #{i+1}', fontsize=12, fontweight='bold')
    plt.tight_layout()
    ruta_arboles_xgb = os.path.join(output_dir, 'estructura_arboles_xgb.png')
    plt.savefig(ruta_arboles_xgb, dpi=300, bbox_inches='tight')
    plt.show()
    print(f'✓ Estructuras XGBoost guardadas en: {ruta_arboles_xgb}', flush=True)
    exitos_xgb = True
except Exception as e:
    print(f'Nota: no fue posible visualizar árboles XGBoost directamente ({e}). Se usa árbol surrogado.', flush=True)

# Intento 2: Árbol surrogado con sklearn sobre top-N variables más importantes
from sklearn.tree import DecisionTreeClassifier, plot_tree as sk_plot_tree

# Tomar top-N variables
top_n = 10
indices_top = []
for feat in df_importancia['Feature'].head(top_n).tolist():
    # 'Feature_i' -> i
    try:
        idx = int(feat.split('_')[1])
    except Exception:
        idx = 0
    indices_top.append(idx)

# Preparar subconjunto de datos (usar escalados por consistencia)
X_train_top = X_train_scaled[:, indices_top]
X_test_top = X_test_scaled[:, indices_top]

arbol_surrogado = DecisionTreeClassifier(max_depth=4, random_state=42)
arbol_surrogado.fit(X_train_top, y_train)

fig = plt.figure(figsize=(24, 14))
sk_plot_tree(
    arbol_surrogado,
    feature_names=[df_importancia['Feature'].iloc[i] for i in range(top_n)],
    class_names=['Clase 0', 'Clase 1'],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title('Árbol de Decisión Surrogado (Top 10 variables)', fontsize=14, fontweight='bold')
ruta_arbol_surrogado = os.path.join(output_dir, 'arbol_surrogado_top10.png')
plt.savefig(ruta_arbol_surrogado, dpi=300, bbox_inches='tight')
plt.show()
print(f'✓ Árbol surrogado guardado en: {ruta_arbol_surrogado}', flush=True)

# Exportar también una versión más pequeña (profundidad 3) para lectura rápida
arbol_surrogado_peq = DecisionTreeClassifier(max_depth=3, random_state=42)
arbol_surrogado_peq.fit(X_train_top, y_train)
fig = plt.figure(figsize=(20, 12))
sk_plot_tree(
    arbol_surrogado_peq,
    feature_names=[df_importancia['Feature'].iloc[i] for i in range(top_n)],
    class_names=['Clase 0', 'Clase 1'],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title('Árbol Surrogado (Profundidad 3) - Top 10 variables', fontsize=14, fontweight='bold')
ruta_arbol_surrogado_peq = os.path.join(output_dir, 'arbol_surrogado_top10_depth3.png')
plt.savefig(ruta_arbol_surrogado_peq, dpi=300, bbox_inches='tight')
plt.show()
print(f'✓ Árbol surrogado (profundidad 3) guardado en: {ruta_arbol_surrogado_peq}', flush=True)


### 12.1: Visualizaciones EDA Completas


In [ ]:
print('=' * 80)
print('VISUALIZACIONES EDA COMPLETAS')
print('=' * 80)

print('\nCargando datos originales para EDA...', flush=True)

archivo = 'Total_Mes_Act_Datos completos CORREGIDO.csv'
df_eda = pd.read_csv(
    archivo, 
    low_memory=False,
    encoding='utf-8',
    on_bad_lines='skip',
    sep=';',
    quotechar='"'
)

print(f'✓ Datos cargados: {df_eda.shape[0]} filas, {df_eda.shape[1]} columnas', flush=True)

columna_tarifa = 'TARIFA_NBO'
columna_rentabilizacion = 'Rentabilizo'

df_eda['variable_objetivo'] = np.where(
    (df_eda[columna_tarifa].notna()) & 
    (df_eda[columna_rentabilizacion].notna()),
    1,
    0
)

fig = plt.figure(figsize=(20, 16))

distribucion = df_eda['variable_objetivo'].value_counts()
ax1 = plt.subplot(3, 3, 1)
distribucion.plot(kind='bar', color=['coral', 'steelblue'], ax=ax1)
ax1.set_title('Distribución de Variable Objetivo', fontsize=12, fontweight='bold', pad=15)
ax1.set_xlabel('Clase (0=No cumple, 1=Cumple)', fontsize=10, fontweight='bold')
ax1.set_ylabel('Frecuencia', fontsize=10, fontweight='bold')
ax1.set_xticklabels(['No cumple', 'Cumple'], rotation=0)
for i, v in enumerate(distribucion.values):
    ax1.text(i, v + max(distribucion.values) * 0.01, f'{v:,}', 
            ha='center', va='bottom', fontweight='bold')

ax2 = plt.subplot(3, 3, 2)
distribucion.plot(kind='pie', autopct='%1.1f%%', colors=['coral', 'steelblue'], ax=ax2, startangle=90)
ax2.set_title('Distribución Porcentual', fontsize=12, fontweight='bold', pad=15)
ax2.set_ylabel('')

missing = df_eda.isnull().sum()
missing_pct = (missing / len(df_eda)) * 100
missing_df = pd.DataFrame({
    'Valores_Faltantes': missing,
    'Porcentaje': missing_pct
})
missing_df = missing_df[missing_df['Valores_Faltantes'] > 0].sort_values('Porcentaje', ascending=False).head(15)

ax3 = plt.subplot(3, 3, 3)
missing_df['Porcentaje'].plot(kind='barh', color='orange', ax=ax3)
ax3.set_title('Top 15 Columnas con Valores Faltantes', fontsize=12, fontweight='bold', pad=15)
ax3.set_xlabel('Porcentaje de Valores Faltantes', fontsize=10, fontweight='bold')
ax3.invert_yaxis()

columnas_numericas = df_eda.select_dtypes(include=[np.number]).columns.tolist()
if len(columnas_numericas) > 0:
    df_numericas = df_eda[columnas_numericas[:6]].copy()
    
    ax4 = plt.subplot(3, 3, 4)
    df_numericas.boxplot(ax=ax4, rot=45)
    ax4.set_title('Distribución de Variables Numéricas (Top 6)', fontsize=12, fontweight='bold', pad=15)
    ax4.set_ylabel('Valor', fontsize=10, fontweight='bold')
    
    ax5 = plt.subplot(3, 3, 5)
    df_numericas.hist(bins=30, ax=ax5, layout=(2, 3), figsize=(15, 10))
    ax5.set_title('Histogramas de Variables Numéricas', fontsize=12, fontweight='bold', pad=15)

if columna_tarifa in df_eda.columns:
    ax6 = plt.subplot(3, 3, 6)
    df_eda[columna_tarifa].notna().value_counts().plot(kind='bar', color=['red', 'green'], ax=ax6)
    ax6.set_title(f'Distribución de {columna_tarifa}', fontsize=12, fontweight='bold', pad=15)
    ax6.set_xlabel('Tiene Valor', fontsize=10, fontweight='bold')
    ax6.set_ylabel('Frecuencia', fontsize=10, fontweight='bold')
    ax6.set_xticklabels(['Faltante', 'Presente'], rotation=0)

if columna_rentabilizacion in df_eda.columns:
    ax7 = plt.subplot(3, 3, 7)
    df_eda[columna_rentabilizacion].notna().value_counts().plot(kind='bar', color=['red', 'green'], ax=ax7)
    ax7.set_title(f'Distribución de {columna_rentabilizacion}', fontsize=12, fontweight='bold', pad=15)
    ax7.set_xlabel('Tiene Valor', fontsize=10, fontweight='bold')
    ax7.set_ylabel('Frecuencia', fontsize=10, fontweight='bold')
    ax7.set_xticklabels(['Faltante', 'Presente'], rotation=0)

tipos_datos = df_eda.dtypes.value_counts()
ax8 = plt.subplot(3, 3, 8)
tipos_datos.plot(kind='bar', color='purple', ax=ax8)
ax8.set_title('Distribución de Tipos de Datos', fontsize=12, fontweight='bold', pad=15)
ax8.set_xlabel('Tipo de Dato', fontsize=10, fontweight='bold')
ax8.set_ylabel('Cantidad de Columnas', fontsize=10, fontweight='bold')
ax8.tick_params(axis='x', rotation=45)

ax9 = plt.subplot(3, 3, 9)
resumen_stats = {
    'Total Filas': len(df_eda),
    'Total Columnas': len(df_eda.columns),
    'Columnas Numéricas': len(columnas_numericas),
    'Columnas Categóricas': len(df_eda.columns) - len(columnas_numericas),
    'Clase 1 (Cumple)': int(distribucion.get(1, 0)),
    'Clase 0 (No cumple)': int(distribucion.get(0, 0))
}
stats_df = pd.DataFrame(list(resumen_stats.items()), columns=['Métrica', 'Valor'])
stats_df.set_index('Métrica')['Valor'].plot(kind='barh', color='teal', ax=ax9)
ax9.set_title('Resumen Estadístico del Dataset', fontsize=12, fontweight='bold', pad=15)
ax9.set_xlabel('Valor', fontsize=10, fontweight='bold')
ax9.invert_yaxis()

plt.tight_layout()
ruta_eda_completo = os.path.join(output_dir, 'eda_visualizaciones_completas.png')
plt.savefig(ruta_eda_completo, dpi=300, bbox_inches='tight')
print(f'\n✓ Visualizaciones EDA completas guardadas en: {ruta_eda_completo}', flush=True)
plt.show()

print('\n✓ Visualizaciones EDA completadas', flush=True)


### 12.2: Árboles de Decisión Individuales con Variables


In [ ]:
print('=' * 80)
print('GENERANDO ÁRBOLES DE DECISIÓN INDIVIDUALES')
print('=' * 80)

print('\nCargando datos y modelo...', flush=True)

if 'df_importancia' not in globals():
    ruta_importancias = os.path.join(output_dir, 'importancia_caracteristicas.csv')
    if os.path.exists(ruta_importancias):
        df_importancia = pd.read_csv(ruta_importancias)
    else:
        raise RuntimeError('Ejecute el Paso 11 primero para generar importancias.')

if 'xgb_optimizado' not in globals():
    ruta_modelo = os.path.join(output_dir, 'modelo_xgb_optimizado.pkl')
    if os.path.exists(ruta_modelo):
        with open(ruta_modelo, 'rb') as f:
            xgb_optimizado = pickle.load(f)
    else:
        raise RuntimeError('Ejecute el Paso 11 primero para generar el modelo.')

if 'X_train_scaled' not in globals() or 'y_train' not in globals():
    ruta_datos = os.path.join(output_dir, 'datos_preparados_7_1.pkl')
    with open(ruta_datos, 'rb') as f:
        datos = pickle.load(f)
    X_train_scaled = datos['X_train_scaled']
    y_train = datos['y_train']

from sklearn.tree import DecisionTreeClassifier, plot_tree

top_n_vars = 10
top_features = df_importancia.head(top_n_vars)['Feature'].tolist()

indices_top = []
for feat in top_features:
    try:
        idx = int(feat.split('_')[1])
        indices_top.append(idx)
    except:
        pass

if len(indices_top) == 0:
    indices_top = list(range(min(top_n_vars, X_train_scaled.shape[1])))

X_train_top = X_train_scaled[:, indices_top]
feature_names_top = [df_importancia.iloc[i]['Feature'] for i in range(len(indices_top))]

print(f'\nGenerando árboles individuales con top {top_n_vars} variables...', flush=True)
print(f'Variables utilizadas: {feature_names_top}', flush=True)

profundidades = [3, 4, 5]
for depth in profundidades:
    arbol = DecisionTreeClassifier(max_depth=depth, random_state=42, min_samples_split=100)
    arbol.fit(X_train_top, y_train)
    
    fig = plt.figure(figsize=(24, 14))
    plot_tree(
        arbol,
        feature_names=feature_names_top,
        class_names=['Clase 0\n(No cumple)', 'Clase 1\n(Cumple)'],
        filled=True,
        rounded=True,
        fontsize=9,
        proportion=True
    )
    plt.title(f'Árbol de Decisión - Profundidad {depth} - Top {top_n_vars} Variables\nVariables: {", ".join(feature_names_top[:5])}...', 
              fontsize=14, fontweight='bold', pad=20)
    
    ruta_arbol = os.path.join(output_dir, f'arbol_decision_depth{depth}_top{top_n_vars}.png')
    plt.savefig(ruta_arbol, dpi=300, bbox_inches='tight')
    print(f'✓ Árbol (profundidad {depth}) guardado en: {ruta_arbol}', flush=True)
    plt.close()

print('\nGenerando árboles individuales por variable más importante...', flush=True)

for i, (idx, row) in enumerate(df_importancia.head(5).iterrows()):
    feat_name = row['Feature']
    try:
        feat_idx = int(feat_name.split('_')[1])
    except:
        feat_idx = i
    
    if feat_idx >= X_train_scaled.shape[1]:
        continue
    
    indices_single = [feat_idx] + [j for j in indices_top[:4] if j != feat_idx]
    X_train_single = X_train_scaled[:, indices_single[:5]]
    feature_names_single = [df_importancia.iloc[j]['Feature'] if j < len(df_importancia) else f'Feature_{j}' 
                           for j in indices_single[:5]]
    
    arbol_single = DecisionTreeClassifier(max_depth=4, random_state=42, min_samples_split=100)
    arbol_single.fit(X_train_single, y_train)
    
    fig = plt.figure(figsize=(22, 12))
    plot_tree(
        arbol_single,
        feature_names=feature_names_single,
        class_names=['Clase 0', 'Clase 1'],
        filled=True,
        rounded=True,
        fontsize=9
    )
    plt.title(f'Árbol #{i+1} - Variable Principal: {feat_name}\nImportancia: {row["Importance"]:.4f} ({row["Importance"]/df_importancia["Importance"].sum()*100:.2f}%)', 
              fontsize=13, fontweight='bold', pad=20)
    
    ruta_arbol_single = os.path.join(output_dir, f'arbol_decision_var_{feat_name}.png')
    plt.savefig(ruta_arbol_single, dpi=300, bbox_inches='tight')
    print(f'✓ Árbol para {feat_name} guardado en: {ruta_arbol_single}', flush=True)
    plt.close()

print('\n✓ Todos los árboles de decisión individuales generados exitosamente', flush=True)


In [ ]:
print('=' * 80)
print('PASO 13: VISUALIZACIONES EDA INDIVIDUALES')
print('=' * 80)

print('\nCargando datos originales para EDA...', flush=True)

archivo = 'Total_Mes_Act_Datos completos CORREGIDO.csv'
df_eda = pd.read_csv(
    archivo, 
    low_memory=False,
    encoding='utf-8',
    on_bad_lines='skip',
    sep=';',
    quotechar='"'
)

print(f'✓ Datos cargados: {df_eda.shape[0]} filas, {df_eda.shape[1]} columnas', flush=True)

columna_tarifa = 'TARIFA_NBO'
columna_rentabilizacion = 'Rentabilizo'

df_eda['variable_objetivo'] = np.where(
    (df_eda[columna_tarifa].notna()) & 
    (df_eda[columna_rentabilizacion].notna()),
    1,
    0
)

distribucion = df_eda['variable_objetivo'].value_counts()

print('\nGenerando visualizaciones individuales...', flush=True)

# 1. Distribución de Variable Objetivo (Barras)
fig, ax = plt.subplots(figsize=(10, 6))
distribucion.plot(kind='bar', color=['coral', 'steelblue'], ax=ax)
ax.set_title('Distribución de Variable Objetivo', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Clase (0=No cumple, 1=Cumple)', fontsize=12, fontweight='bold')
ax.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
ax.set_xticklabels(['No cumple', 'Cumple'], rotation=0)
for i, v in enumerate(distribucion.values):
    ax.text(i, v + max(distribucion.values) * 0.01, f'{v:,}', 
            ha='center', va='bottom', fontweight='bold', fontsize=11)
plt.tight_layout()
ruta = os.path.join(output_dir, 'eda_01_distribucion_variable_objetivo.png')
plt.savefig(ruta, dpi=300, bbox_inches='tight')
plt.close()
print(f'✓ 1. Distribución variable objetivo guardada en: {ruta}', flush=True)

# 2. Distribución Porcentual (Pastel)
fig, ax = plt.subplots(figsize=(10, 8))
distribucion.plot(kind='pie', autopct='%1.1f%%', colors=['coral', 'steelblue'], ax=ax, startangle=90)
ax.set_title('Distribución Porcentual de Variable Objetivo', fontsize=14, fontweight='bold', pad=20)
ax.set_ylabel('')
plt.tight_layout()
ruta = os.path.join(output_dir, 'eda_02_distribucion_porcentual.png')
plt.savefig(ruta, dpi=300, bbox_inches='tight')
plt.close()
print(f'✓ 2. Distribución porcentual guardada en: {ruta}', flush=True)

# 3. Top 15 Columnas con Valores Faltantes
missing = df_eda.isnull().sum()
missing_pct = (missing / len(df_eda)) * 100
missing_df = pd.DataFrame({
    'Valores_Faltantes': missing,
    'Porcentaje': missing_pct
})
missing_df = missing_df[missing_df['Valores_Faltantes'] > 0].sort_values('Porcentaje', ascending=False).head(15)

if len(missing_df) > 0:
    fig, ax = plt.subplots(figsize=(12, 8))
    missing_df['Porcentaje'].plot(kind='barh', color='orange', ax=ax)
    ax.set_title('Top 15 Columnas con Valores Faltantes', fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Porcentaje de Valores Faltantes', fontsize=12, fontweight='bold')
    ax.set_ylabel('Columna', fontsize=12, fontweight='bold')
    ax.invert_yaxis()
    for i, v in enumerate(missing_df['Porcentaje'].values):
        ax.text(v + 0.5, i, f'{v:.2f}%', va='center', fontweight='bold', fontsize=9)
    plt.tight_layout()
    ruta = os.path.join(output_dir, 'eda_03_valores_faltantes.png')
    plt.savefig(ruta, dpi=300, bbox_inches='tight')
    plt.close()
    print(f'✓ 3. Valores faltantes guardada en: {ruta}', flush=True)

# 4. Boxplots de Variables Numéricas
columnas_numericas = df_eda.select_dtypes(include=[np.number]).columns.tolist()
if len(columnas_numericas) > 0:
    df_numericas = df_eda[columnas_numericas[:6]].copy()
    fig, ax = plt.subplots(figsize=(14, 8))
    df_numericas.boxplot(ax=ax, rot=45)
    ax.set_title('Distribución de Variables Numéricas (Top 6)', fontsize=14, fontweight='bold', pad=20)
    ax.set_ylabel('Valor', fontsize=12, fontweight='bold')
    plt.tight_layout()
    ruta = os.path.join(output_dir, 'eda_04_boxplots_variables_numericas.png')
    plt.savefig(ruta, dpi=300, bbox_inches='tight')
    plt.close()
    print(f'✓ 4. Boxplots variables numéricas guardada en: {ruta}', flush=True)

# 5. Histogramas de Variables Numéricas
if len(columnas_numericas) > 0:
    df_numericas = df_eda[columnas_numericas[:6]].copy()
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    for i, col in enumerate(df_numericas.columns):
        if i < len(axes):
            df_numericas[col].hist(bins=30, ax=axes[i], color='steelblue', edgecolor='black')
            axes[i].set_title(f'{col}', fontsize=11, fontweight='bold')
            axes[i].set_xlabel('Valor', fontsize=10)
            axes[i].set_ylabel('Frecuencia', fontsize=10)
            axes[i].grid(alpha=0.3)
    plt.suptitle('Histogramas de Variables Numéricas (Top 6)', fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    ruta = os.path.join(output_dir, 'eda_05_histogramas_variables_numericas.png')
    plt.savefig(ruta, dpi=300, bbox_inches='tight')
    plt.close()
    print(f'✓ 5. Histogramas variables numéricas guardada en: {ruta}', flush=True)

# 6. Distribución de TARIFA_NBO
if columna_tarifa in df_eda.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    df_eda[columna_tarifa].notna().value_counts().plot(kind='bar', color=['red', 'green'], ax=ax)
    ax.set_title(f'Distribución de {columna_tarifa}', fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Tiene Valor', fontsize=12, fontweight='bold')
    ax.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
    ax.set_xticklabels(['Faltante', 'Presente'], rotation=0)
    for i, v in enumerate(df_eda[columna_tarifa].notna().value_counts().values):
        ax.text(i, v + max(df_eda[columna_tarifa].notna().value_counts().values) * 0.01, f'{v:,}', 
                ha='center', va='bottom', fontweight='bold', fontsize=11)
    plt.tight_layout()
    ruta = os.path.join(output_dir, 'eda_06_distribucion_tarifa_nbo.png')
    plt.savefig(ruta, dpi=300, bbox_inches='tight')
    plt.close()
    print(f'✓ 6. Distribución TARIFA_NBO guardada en: {ruta}', flush=True)

# 7. Distribución de Rentabilizo
if columna_rentabilizacion in df_eda.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    df_eda[columna_rentabilizacion].notna().value_counts().plot(kind='bar', color=['red', 'green'], ax=ax)
    ax.set_title(f'Distribución de {columna_rentabilizacion}', fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Tiene Valor', fontsize=12, fontweight='bold')
    ax.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
    ax.set_xticklabels(['Faltante', 'Presente'], rotation=0)
    for i, v in enumerate(df_eda[columna_rentabilizacion].notna().value_counts().values):
        ax.text(i, v + max(df_eda[columna_rentabilizacion].notna().value_counts().values) * 0.01, f'{v:,}', 
                ha='center', va='bottom', fontweight='bold', fontsize=11)
    plt.tight_layout()
    ruta = os.path.join(output_dir, 'eda_07_distribucion_rentabilizo.png')
    plt.savefig(ruta, dpi=300, bbox_inches='tight')
    plt.close()
    print(f'✓ 7. Distribución Rentabilizo guardada en: {ruta}', flush=True)

# 8. Distribución de Tipos de Datos
tipos_datos = df_eda.dtypes.value_counts()
fig, ax = plt.subplots(figsize=(10, 6))
tipos_datos.plot(kind='bar', color='purple', ax=ax)
ax.set_title('Distribución de Tipos de Datos', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Tipo de Dato', fontsize=12, fontweight='bold')
ax.set_ylabel('Cantidad de Columnas', fontsize=12, fontweight='bold')
ax.tick_params(axis='x', rotation=45)
for i, v in enumerate(tipos_datos.values):
    ax.text(i, v + max(tipos_datos.values) * 0.01, str(v), 
            ha='center', va='bottom', fontweight='bold', fontsize=11)
plt.tight_layout()
ruta = os.path.join(output_dir, 'eda_08_distribucion_tipos_datos.png')
plt.savefig(ruta, dpi=300, bbox_inches='tight')
plt.close()
print(f'✓ 8. Distribución tipos de datos guardada en: {ruta}', flush=True)

# 9. Resumen Estadístico del Dataset
resumen_stats = {
    'Total Filas': len(df_eda),
    'Total Columnas': len(df_eda.columns),
    'Columnas Numéricas': len(columnas_numericas),
    'Columnas Categóricas': len(df_eda.columns) - len(columnas_numericas),
    'Clase 1 (Cumple)': int(distribucion.get(1, 0)),
    'Clase 0 (No cumple)': int(distribucion.get(0, 0))
}
stats_df = pd.DataFrame(list(resumen_stats.items()), columns=['Métrica', 'Valor'])
fig, ax = plt.subplots(figsize=(12, 8))
stats_df.set_index('Métrica')['Valor'].plot(kind='barh', color='teal', ax=ax)
ax.set_title('Resumen Estadístico del Dataset', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Valor', fontsize=12, fontweight='bold')
ax.set_ylabel('Métrica', fontsize=12, fontweight='bold')
ax.invert_yaxis()
for i, v in enumerate(stats_df['Valor'].values):
    ax.text(v + max(stats_df['Valor'].values) * 0.01, i, f'{v:,}', 
            va='center', fontweight='bold', fontsize=11)
plt.tight_layout()
ruta = os.path.join(output_dir, 'eda_09_resumen_estadistico.png')
plt.savefig(ruta, dpi=300, bbox_inches='tight')
plt.close()
print(f'✓ 9. Resumen estadístico guardada en: {ruta}', flush=True)

print('\n✓ Paso 13 completado exitosamente')
print(f'\nTodas las visualizaciones EDA individuales están guardadas en: {output_dir}/', flush=True)


## Paso 14: Comparación de Modelos (Regresión Logística, Random Forest, XGBoost)

En este paso entrenamos los tres modelos principales usando los datos donde TARIFA_NBO y Rentabilizo tienen información, generamos visualizaciones comparativas y validamos cuál modelo es el mejor.


In [ ]:
print('=' * 80)
print('PASO 14: COMPARACIÓN DE MODELOS')
print('=' * 80)

# Crear subcarpeta para archivos del paso 14
output_dir_paso14 = os.path.join(output_dir, 'paso14')
os.makedirs(output_dir_paso14, exist_ok=True)
print(f'\n✓ Subcarpeta creada: {output_dir_paso14}', flush=True)

print('\nCargando y preparando datos...', flush=True)

archivo = 'Total_Mes_Act_Datos completos CORREGIDO.csv'
df = pd.read_csv(
    archivo, 
    low_memory=False,
    encoding='utf-8',
    on_bad_lines='skip',
    sep=';',
    quotechar='"'
)

print(f'✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas', flush=True)

columna_tarifa = 'TARIFA_NBO'
columna_rentabilizacion = 'Rentabilizo'

df['variable_objetivo'] = np.where(
    (df[columna_tarifa].notna()) & 
    (df[columna_rentabilizacion].notna()),
    1,
    0
)

y = df['variable_objetivo'].copy()
X = df.drop(columns=['variable_objetivo', columna_tarifa, columna_rentabilizacion])

print(f'\nVariable objetivo creada:', flush=True)
print(f'  Clase 0 (No cumple): {sum(y == 0):,} registros ({sum(y == 0)/len(y)*100:.2f}%)', flush=True)
print(f'  Clase 1 (Cumple): {sum(y == 1):,} registros ({sum(y == 1)/len(y)*100:.2f}%)', flush=True)

columnas_numericas = X.select_dtypes(include=[np.number]).columns.tolist()
columnas_categoricas = X.select_dtypes(include=['object', 'category']).columns.tolist()

X_processed = X[columnas_numericas].copy()

print(f'\nProcesando {len(columnas_categoricas)} columnas categóricas...', flush=True)
for i, col in enumerate(columnas_categoricas):
    if X[col].nunique() < 50:
        le = LabelEncoder()
        X_processed[col] = le.fit_transform(X[col].astype(str).fillna('Missing'))
    if (i + 1) % 10 == 0:
        print(f'  Procesadas {i + 1}/{len(columnas_categoricas)} columnas...', flush=True)

X_processed = X_processed.fillna(X_processed.median())

# Guardar nombres de columnas originales para mapeo
feature_names = list(X_processed.columns)

print(f'\n✓ Datos procesados: {X_processed.shape[1]} features', flush=True)
print(f'✓ Nombres de features guardados para mapeo', flush=True)

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

print(f'✓ División train/test:', flush=True)
print(f'  Train: {X_train.shape[0]:,} muestras', flush=True)
print(f'  Test: {X_test.shape[0]:,} muestras', flush=True)


### 14.1: Entrenamiento de Modelos


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                            f1_score, roc_auc_score, confusion_matrix, 
                            classification_report, roc_curve, auc, 
                            precision_recall_curve)

print('\n' + '=' * 80)
print('ENTRENAMIENTO DE MODELOS')
print('=' * 80)

modelos = {}
resultados_modelos = {}

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('\n1. Regresión Logística...', flush=True)
lr = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
y_pred_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]

modelos['LogisticRegression'] = lr
resultados_modelos['LogisticRegression'] = {
    'y_pred': y_pred_lr,
    'y_pred_proba': y_pred_proba_lr,
    'scaler': scaler
}
print('  ✓ Regresión Logística entrenada', flush=True)

print('\n2. Random Forest...', flush=True)
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=20)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_pred_proba_rf = rf.predict_proba(X_test)[:, 1]

modelos['RandomForest'] = rf
resultados_modelos['RandomForest'] = {
    'y_pred': y_pred_rf,
    'y_pred_proba': y_pred_proba_rf,
    'scaler': None
}
print('  ✓ Random Forest entrenado', flush=True)

print('\n3. XGBoost...', flush=True)
xgb = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, 
                    random_state=42, n_jobs=-1, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
y_pred_proba_xgb = xgb.predict_proba(X_test)[:, 1]

modelos['XGBoost'] = xgb
resultados_modelos['XGBoost'] = {
    'y_pred': y_pred_xgb,
    'y_pred_proba': y_pred_proba_xgb,
    'scaler': None
}
print('  ✓ XGBoost entrenado', flush=True)

print('\n✓ Todos los modelos entrenados exitosamente', flush=True)


### 14.2: Cálculo de Métricas y Comparación


In [ ]:
print('\n' + '=' * 80)
print('CÁLCULO DE MÉTRICAS')
print('=' * 80)

metricas_comparacion = []

for nombre, modelo in modelos.items():
    y_pred = resultados_modelos[nombre]['y_pred']
    y_pred_proba = resultados_modelos[nombre]['y_pred_proba']
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
    pr_auc = auc(recall_curve, precision_curve)
    
    metricas_comparacion.append({
        'Modelo': nombre,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    })
    
    print(f'\n{nombre}:', flush=True)
    print(f'  Accuracy:  {accuracy:.4f}', flush=True)
    print(f'  Precision: {precision:.4f}', flush=True)
    print(f'  Recall:    {recall:.4f}', flush=True)
    print(f'  F1-Score:  {f1:.4f}', flush=True)
    print(f'  ROC-AUC:   {roc_auc:.4f}', flush=True)
    print(f'  PR-AUC:    {pr_auc:.4f}', flush=True)

df_metricas = pd.DataFrame(metricas_comparacion)
df_metricas.to_csv(os.path.join(output_dir_paso14, 'comparacion_modelos_metricas.csv'), index=False)
print(f'\n✓ Métricas guardadas en: comparacion_modelos_metricas.csv', flush=True)

mejor_modelo_accuracy = df_metricas.loc[df_metricas['Accuracy'].idxmax(), 'Modelo']
mejor_modelo_f1 = df_metricas.loc[df_metricas['F1-Score'].idxmax(), 'Modelo']
mejor_modelo_roc = df_metricas.loc[df_metricas['ROC-AUC'].idxmax(), 'Modelo']

print('\n' + '=' * 80)
print('MEJOR MODELO POR MÉTRICA')
print('=' * 80)
print(f'  Mejor Accuracy:  {mejor_modelo_accuracy} ({df_metricas.loc[df_metricas["Accuracy"].idxmax(), "Accuracy"]:.4f})', flush=True)
print(f'  Mejor F1-Score:  {mejor_modelo_f1} ({df_metricas.loc[df_metricas["F1-Score"].idxmax(), "F1-Score"]:.4f})', flush=True)
print(f'  Mejor ROC-AUC:   {mejor_modelo_roc} ({df_metricas.loc[df_metricas["ROC-AUC"].idxmax(), "ROC-AUC"]:.4f})', flush=True)


### 14.3: Visualizaciones Comparativas


In [ ]:
print('\n' + '=' * 80)
print('GENERANDO VISUALIZACIONES COMPARATIVAS')
print('=' * 80)

fig = plt.figure(figsize=(20, 14))

# 1. Comparación de Métricas (Barras)
ax1 = plt.subplot(2, 3, 1)
metricas_plot = df_metricas.set_index('Modelo')[['Accuracy', 'Precision', 'Recall', 'F1-Score']]
metricas_plot.plot(kind='bar', ax=ax1, width=0.8)
ax1.set_title('Comparación de Métricas Básicas', fontsize=12, fontweight='bold', pad=15)
ax1.set_ylabel('Score', fontsize=10, fontweight='bold')
ax1.set_xlabel('Modelo', fontsize=10, fontweight='bold')
ax1.legend(loc='upper left', fontsize=9)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# 2. Comparación ROC-AUC y PR-AUC
ax2 = plt.subplot(2, 3, 2)
auc_plot = df_metricas.set_index('Modelo')[['ROC-AUC', 'PR-AUC']]
auc_plot.plot(kind='bar', ax=ax2, width=0.8, color=['steelblue', 'coral'])
ax2.set_title('Comparación ROC-AUC y PR-AUC', fontsize=12, fontweight='bold', pad=15)
ax2.set_ylabel('AUC Score', fontsize=10, fontweight='bold')
ax2.set_xlabel('Modelo', fontsize=10, fontweight='bold')
ax2.legend(loc='upper left', fontsize=9)
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)

# 3. Curvas ROC Comparativas
ax3 = plt.subplot(2, 3, 3)
for nombre in modelos.keys():
    y_pred_proba = resultados_modelos[nombre]['y_pred_proba']
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    ax3.plot(fpr, tpr, lw=2, label=f'{nombre} (AUC = {roc_auc:.4f})')
ax3.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
ax3.set_xlabel('Tasa de Falsos Positivos', fontsize=10, fontweight='bold')
ax3.set_ylabel('Tasa de Verdaderos Positivos', fontsize=10, fontweight='bold')
ax3.set_title('Curvas ROC Comparativas', fontsize=12, fontweight='bold', pad=15)
ax3.legend(loc="lower right", fontsize=9)
ax3.grid(alpha=0.3)

# 4. Curvas Precision-Recall Comparativas
ax4 = plt.subplot(2, 3, 4)
for nombre in modelos.keys():
    y_pred_proba = resultados_modelos[nombre]['y_pred_proba']
    precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
    pr_auc = auc(recall_curve, precision_curve)
    ax4.plot(recall_curve, precision_curve, lw=2, label=f'{nombre} (AUC = {pr_auc:.4f})')
baseline = len(y_test[y_test==1]) / len(y_test)
ax4.axhline(y=baseline, color='r', linestyle='--', label=f'Baseline ({baseline:.4f})')
ax4.set_xlabel('Recall', fontsize=10, fontweight='bold')
ax4.set_ylabel('Precision', fontsize=10, fontweight='bold')
ax4.set_title('Curvas Precision-Recall Comparativas', fontsize=12, fontweight='bold', pad=15)
ax4.legend(loc="lower left", fontsize=9)
ax4.grid(alpha=0.3)

plt.tight_layout()
ruta_comparacion = os.path.join(output_dir_paso14, 'comparacion_modelos_visualizacion.png')
plt.savefig(ruta_comparacion, dpi=300, bbox_inches='tight')
print(f'\n✓ Visualización comparativa guardada en: {ruta_comparacion}', flush=True)
plt.close()

# Matrices de Confusión en figura separada
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for idx, nombre in enumerate(modelos.keys()):
    y_pred = resultados_modelos[nombre]['y_pred']
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx], 
                xticklabels=['Clase 0', 'Clase 1'], 
                yticklabels=['Clase 0', 'Clase 1'])
    axes[idx].set_title(f'Matriz Confusión - {nombre}', fontsize=12, fontweight='bold', pad=15)
    axes[idx].set_xlabel('Predicción', fontsize=10, fontweight='bold')
    axes[idx].set_ylabel('Real', fontsize=10, fontweight='bold')

plt.tight_layout()
ruta_confusion = os.path.join(output_dir_paso14, 'comparacion_modelos_matrices_confusion.png')
plt.savefig(ruta_confusion, dpi=300, bbox_inches='tight')
print(f'✓ Matrices de confusión guardadas en: {ruta_confusion}', flush=True)
plt.close()


### 14.4: Validación Cruzada y Selección del Mejor Modelo


In [ ]:
print('\n' + '=' * 80)
print('VALIDACIÓN CRUZADA (5 FOLDS)')
print('=' * 80)

from sklearn.model_selection import cross_val_score, StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

resultados_cv = {}

print('\nRealizando validación cruzada para cada modelo...', flush=True)

# Regresión Logística
print('\n1. Regresión Logística...', flush=True)
cv_scores_lr = cross_val_score(lr, X_train_scaled, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
resultados_cv['LogisticRegression'] = {
    'mean': cv_scores_lr.mean(),
    'std': cv_scores_lr.std(),
    'scores': cv_scores_lr.tolist()
}
print(f'  Accuracy CV: {cv_scores_lr.mean():.4f} (+/- {cv_scores_lr.std() * 2:.4f})', flush=True)

# Random Forest
print('\n2. Random Forest...', flush=True)
cv_scores_rf = cross_val_score(rf, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
resultados_cv['RandomForest'] = {
    'mean': cv_scores_rf.mean(),
    'std': cv_scores_rf.std(),
    'scores': cv_scores_rf.tolist()
}
print(f'  Accuracy CV: {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std() * 2:.4f})', flush=True)

# XGBoost
print('\n3. XGBoost...', flush=True)
cv_scores_xgb = cross_val_score(xgb, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
resultados_cv['XGBoost'] = {
    'mean': cv_scores_xgb.mean(),
    'std': cv_scores_xgb.std(),
    'scores': cv_scores_xgb.tolist()
}
print(f'  Accuracy CV: {cv_scores_xgb.mean():.4f} (+/- {cv_scores_xgb.std() * 2:.4f})', flush=True)

# Guardar resultados CV
ruta_cv = os.path.join(output_dir_paso14, 'resultados_validacion_cruzada_paso14.json')
with open(ruta_cv, 'w', encoding='utf-8') as f:
    json.dump(resultados_cv, f, indent=2)
print(f'\n✓ Resultados de validación cruzada guardados en: {ruta_cv}', flush=True)

# Visualización de resultados CV
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Boxplot de scores CV
cv_data = [resultados_cv['LogisticRegression']['scores'],
           resultados_cv['RandomForest']['scores'],
           resultados_cv['XGBoost']['scores']]
axes[0].boxplot(cv_data, labels=['Logistic\nRegression', 'Random\nForest', 'XGBoost'])
axes[0].set_title('Distribución de Accuracy en Validación Cruzada', fontsize=14, fontweight='bold', pad=20)
axes[0].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Comparación de medias
medias_cv = [resultados_cv['LogisticRegression']['mean'],
             resultados_cv['RandomForest']['mean'],
             resultados_cv['XGBoost']['mean']]
stds_cv = [resultados_cv['LogisticRegression']['std'],
           resultados_cv['RandomForest']['std'],
           resultados_cv['XGBoost']['std']]
axes[1].bar(['Logistic\nRegression', 'Random\nForest', 'XGBoost'], medias_cv, 
            yerr=[std * 2 for std in stds_cv], capsize=10, color=['steelblue', 'coral', 'green'], alpha=0.7)
axes[1].set_title('Accuracy Promedio en Validación Cruzada', fontsize=14, fontweight='bold', pad=20)
axes[1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)
for i, (media, std) in enumerate(zip(medias_cv, stds_cv)):
    axes[1].text(i, media + std * 2 + 0.01, f'{media:.4f}', 
                ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
ruta_cv_viz = os.path.join(output_dir_paso14, 'validacion_cruzada_paso14.png')
plt.savefig(ruta_cv_viz, dpi=300, bbox_inches='tight')
print(f'✓ Visualización de validación cruzada guardada en: {ruta_cv_viz}', flush=True)
plt.close()

# Determinar mejor modelo
mejor_modelo_cv = max(resultados_cv.items(), key=lambda x: x[1]['mean'])[0]
mejor_score_cv = resultados_cv[mejor_modelo_cv]['mean']

print('\n' + '=' * 80)
print('CONCLUSIÓN FINAL')
print('=' * 80)
print(f'\nMejor modelo según Validación Cruzada: {mejor_modelo_cv}', flush=True)
print(f'  Accuracy promedio: {mejor_score_cv:.4f} (+/- {resultados_cv[mejor_modelo_cv]["std"] * 2:.4f})', flush=True)

print('\nResumen de todos los modelos:', flush=True)
for nombre, resultados in resultados_cv.items():
    print(f'  {nombre:20s}: {resultados["mean"]:.4f} (+/- {resultados["std"] * 2:.4f})', flush=True)

conclusion_final = {
    'mejor_modelo_cv': mejor_modelo_cv,
    'mejor_score_cv': mejor_score_cv,
    'resultados_cv': resultados_cv,
    'metricas_test': metricas_comparacion,
    'mejor_modelo_accuracy': mejor_modelo_accuracy,
    'mejor_modelo_f1': mejor_modelo_f1,
    'mejor_modelo_roc': mejor_modelo_roc
}

ruta_conclusion = os.path.join(output_dir_paso14, 'conclusion_paso14.json')
with open(ruta_conclusion, 'w', encoding='utf-8') as f:
    json.dump(conclusion_final, f, indent=2)
print(f'\n✓ Conclusión final guardada en: {ruta_conclusion}', flush=True)

print('\n✓ Paso 14 completado exitosamente', flush=True)

In [ ]:
print('\n' + '=' * 80)
print('GENERANDO ÁRBOLES DE DECISIÓN CON VARIABLES REALES')
print('=' * 80)

from sklearn.tree import DecisionTreeClassifier, plot_tree

print('\nGenerando árboles de decisión para Random Forest y XGBoost...', flush=True)
print('(Nota: Logistic Regression es un modelo lineal, no genera árboles)', flush=True)

# Obtener importancia de features para cada modelo
print('\n1. Analizando importancia de variables...', flush=True)

# Random Forest - Importancia de features
importancia_rf = rf.feature_importances_
indices_rf = np.argsort(importancia_rf)[::-1]
top_features_rf = indices_rf[:15]  # Top 15 features

# XGBoost - Importancia de features
importancia_xgb = xgb.feature_importances_
indices_xgb = np.argsort(importancia_xgb)[::-1]
top_features_xgb = indices_xgb[:15]  # Top 15 features

print(f'  ✓ Top 15 features identificados para Random Forest', flush=True)
print(f'  ✓ Top 15 features identificados para XGBoost', flush=True)

# Función para obtener nombres reales de features
def obtener_nombres_reales(indices, feature_names):
    return [feature_names[i] if i < len(feature_names) else f'Feature_{i}' for i in indices]

# Generar árboles para Random Forest
print('\n2. Generando árboles de decisión para Random Forest...', flush=True)

# Árbol individual de Random Forest (primer árbol del ensemble)
arbol_rf_individual = rf.estimators_[0]  # Primer árbol del Random Forest

# Árbol completo del Random Forest (usando todos los features importantes)
profundidades_rf = [3, 4, 5]
for depth in profundidades_rf:
    # Usar top features para crear un árbol más interpretable
    if isinstance(X_train, pd.DataFrame):
        X_train_top_rf = X_train.iloc[:, top_features_rf[:10]]
    else:
        X_train_top_rf = X_train[:, top_features_rf[:10]]
    feature_names_top_rf = obtener_nombres_reales(top_features_rf[:10], feature_names)
    
    arbol_rf_surrogado = DecisionTreeClassifier(max_depth=depth, random_state=42, min_samples_split=100)
    arbol_rf_surrogado.fit(X_train_top_rf, y_train)
    
    fig = plt.figure(figsize=(24, 14))
    plot_tree(
        arbol_rf_surrogado,
        feature_names=feature_names_top_rf,
        class_names=['Clase 0\n(No cumple)', 'Clase 1\n(Cumple)'],
        filled=True,
        rounded=True,
        fontsize=9,
        proportion=True
    )
    plt.title(f'Árbol de Decisión - Random Forest - Profundidad {depth}\nTop 10 Variables Más Importantes', 
              fontsize=14, fontweight='bold', pad=20)
    
    ruta_arbol = os.path.join(output_dir_paso14, f'arbol_random_forest_depth{depth}.png')
    plt.savefig(ruta_arbol, dpi=300, bbox_inches='tight')
    plt.close()
    print(f'  ✓ Árbol Random Forest (profundidad {depth}) guardado en: {ruta_arbol}', flush=True)

# Árbol individual del primer estimador de Random Forest
print('\n3. Generando árbol individual del primer estimador de Random Forest...', flush=True)
fig = plt.figure(figsize=(24, 14))
plot_tree(
    arbol_rf_individual,
    feature_names=feature_names,
    class_names=['Clase 0\n(No cumple)', 'Clase 1\n(Cumple)'],
    filled=True,
    rounded=True,
    fontsize=8,
    max_depth=4  # Limitar profundidad para legibilidad
)
plt.title('Árbol Individual - Random Forest (Primer Estimador)\nProfundidad limitada a 4 niveles', 
          fontsize=14, fontweight='bold', pad=20)
ruta_arbol_rf_individual = os.path.join(output_dir_paso14, 'arbol_random_forest_individual.png')
plt.savefig(ruta_arbol_rf_individual, dpi=300, bbox_inches='tight')
plt.close()
print(f'  ✓ Árbol individual guardado en: {ruta_arbol_rf_individual}', flush=True)

# Generar árboles para XGBoost
print('\n4. Generando árboles de decisión para XGBoost...', flush=True)

# XGBoost tiene árboles individuales, pero son más complejos
# Crearemos árboles surrogados basados en las variables más importantes
profundidades_xgb = [3, 4, 5]
for depth in profundidades_xgb:
    # Usar top features para crear un árbol más interpretable
    if isinstance(X_train, pd.DataFrame):
        X_train_top_xgb = X_train.iloc[:, top_features_xgb[:10]]
    else:
        X_train_top_xgb = X_train[:, top_features_xgb[:10]]
    feature_names_top_xgb = obtener_nombres_reales(top_features_xgb[:10], feature_names)
    
    arbol_xgb_surrogado = DecisionTreeClassifier(max_depth=depth, random_state=42, min_samples_split=100)
    arbol_xgb_surrogado.fit(X_train_top_xgb, y_train)
    
    fig = plt.figure(figsize=(24, 14))
    plot_tree(
        arbol_xgb_surrogado,
        feature_names=feature_names_top_xgb,
        class_names=['Clase 0\n(No cumple)', 'Clase 1\n(Cumple)'],
        filled=True,
        rounded=True,
        fontsize=9,
        proportion=True
    )
    plt.title(f'Árbol de Decisión - XGBoost - Profundidad {depth}\nTop 10 Variables Más Importantes', 
              fontsize=14, fontweight='bold', pad=20)
    
    ruta_arbol = os.path.join(output_dir_paso14, f'arbol_xgboost_depth{depth}.png')
    plt.savefig(ruta_arbol, dpi=300, bbox_inches='tight')
    plt.close()
    print(f'  ✓ Árbol XGBoost (profundidad {depth}) guardado en: {ruta_arbol}', flush=True)

# Guardar mapeo de features
mapeo_features = {
    'feature_names': feature_names,
    'top_features_rf': top_features_rf[:15].tolist(),
    'top_features_xgb': top_features_xgb[:15].tolist(),
    'nombres_top_rf': obtener_nombres_reales(top_features_rf[:15], feature_names),
    'nombres_top_xgb': obtener_nombres_reales(top_features_xgb[:15], feature_names)
}

ruta_mapeo = os.path.join(output_dir_paso14, 'mapeo_features_paso14.json')
with open(ruta_mapeo, 'w', encoding='utf-8') as f:
    json.dump(mapeo_features, f, indent=2)
print(f'\n✓ Mapeo de features guardado en: {ruta_mapeo}', flush=True)

# Guardar importancia de features
df_importancia_rf = pd.DataFrame({
    'Feature_Index': top_features_rf[:15],
    'Feature_Name': obtener_nombres_reales(top_features_rf[:15], feature_names),
    'Importance': importancia_rf[top_features_rf[:15]]
})
df_importancia_rf.to_csv(os.path.join(output_dir_paso14, 'importancia_features_random_forest.csv'), index=False)

df_importancia_xgb = pd.DataFrame({
    'Feature_Index': top_features_xgb[:15],
    'Feature_Name': obtener_nombres_reales(top_features_xgb[:15], feature_names),
    'Importance': importancia_xgb[top_features_xgb[:15]]
})
df_importancia_xgb.to_csv(os.path.join(output_dir_paso14, 'importancia_features_xgboost.csv'), index=False)

print(f'✓ Importancia de features guardada en CSV', flush=True)

print('\n' + '=' * 80)
print('RESUMEN DE ÁRBOLES GENERADOS')
print('=' * 80)
print(f'\nRandom Forest:')
print(f'  - Árbol individual (profundidad 4): arbol_random_forest_individual.png')
for depth in profundidades_rf:
    print(f'  - Árbol surrogado (profundidad {depth}): arbol_random_forest_depth{depth}.png')

print(f'\nXGBoost:')
for depth in profundidades_xgb:
    print(f'  - Árbol surrogado (profundidad {depth}): arbol_xgboost_depth{depth}.png')

print(f'\n✓ Todos los árboles guardados en: {output_dir_paso14}/', flush=True)
print('\n✓ Generación de árboles completada exitosamente', flush=True)